In [ ]:
import gc
import json
import math
import os
import pickle
import random
import re
import sys

import numpy as np
from charset_normalizer import from_path
from gensim.corpora import Dictionary
import nltk
from tqdm import tqdm

"""
Se importan las librerias que se necesiten, 
si se quiere ejecutar el notebook, se recomienda crear la carpeta de data, y poner ahi los files como se describe

"""
ACTUAL_PATH = os.getcwd()
# Donde esta el 20 News
PATH_20N = os.path.join(ACTUAL_PATH, "data/20news-18828")
# Donde se encuentra el BAC
PATH_BAC = os.path.join(ACTUAL_PATH, "data/BAC/blogs")
# Donde se van a guardar los files que se van obteniendo
PATH_FINAL_FILES = os.path.join(ACTUAL_PATH, "data/final_files")

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
import nltk
from nltk.corpus import stopwords
import re
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer


stemmer = nltk.stem.SnowballStemmer("english")
nltk.download("stopwords")

### Upload 20N 

In [2]:
"""Para esto se define en que formato y donde se quiere el archivo completo de 20N"""

NEW_20N_FILE = os.path.join(PATH_FINAL_FILES, "20N.jsonl")

mayor_folders_20N = os.listdir(PATH_20N)
dictionary = {}
"""
Para cada archivo disponible en 20N,  
se generan registros con:  
- el ID del archivo  
- el tema del archivo  
- el texto del contenido  

Todo se guarda en formato JSONL por cuestiones de formato.
"""
with open(NEW_20N_FILE, "w", encoding="utf-8", errors="replace") as f_n:
    for folder in mayor_folders_20N:
        minor_files_path = os.path.join(PATH_20N, folder)
        minor_files = os.listdir(minor_files_path)
        for file in minor_files:
            file_path = os.path.join(minor_files_path, file)
            with open(file_path, "r", encoding="utf-8", errors="replace") as f:
                text = f.read().strip()

            record = {"id": file, "theme": folder, "text": text}
            unit = folder + file
            if file in dictionary.keys():
                dictionary[unit] += 1
            else:
                dictionary[unit] = 1
            f_n.write(json.dumps(record, ensure_ascii=False) + "\n")

In [28]:
from nltk.stem import SnowballStemmer
from nltk.corpus import stopwords


stemmer = SnowballStemmer("english")
stop_words = stopwords.words("english")


def preprocess_text(text: str) -> list[str]:
    """Limpia y preprocesa texto: elimina correos, URLs,
    normaliza, tokeniza y aplica stemming (para inglés)."""

    text = text.lower()

    text = re.sub(r"\b[A-Za-z0-9._%+-]+@[A-Za-z0-9.-]+\.[A-Z|a-z]{2,}\b", " ", text)

    text = re.sub(r"http\S+|www\.\S+", " ", text)

    text = re.sub(r"\([^)]*\)", " ", text)

    text = re.sub(r"\d+", " NUM ", text)

    text = re.sub(r"[^a-z0-9'\-]", " ", text)

    text = re.sub(r"\s+", " ", text).strip()

    tokens = nltk.word_tokenize(text, language="english")

    tokens = [
        stemmer.stem(token)
        for token in tokens
        if token.isalpha() and token not in stop_words
    ]

    return tokens


def extract_body(text: str) -> str:
    """Extrae el cuerpo del texto dejando el Subject al inicio y eliminando From."""

    text = re.sub(r"^From:.*\n", "", text, flags=re.MULTILINE)

    subject_match = re.search(
        r"^Subject:\s*(.*)", text, flags=re.MULTILINE | re.IGNORECASE
    )
    subject = subject_match.group(1).strip() if subject_match else ""

    body = re.sub(
        r"^Subject:.*\n", "", text, flags=re.MULTILINE | re.IGNORECASE
    ).strip()

    if subject:
        body = subject + "\n\n" + body

    return body

In [ ]:

texts = []
labels = []

with open(os.path.join(PATH_FINAL_FILES, "20N.jsonl")) as f:
    for line in f:
        data = json.loads(line)
        labels.append(data["theme"])      
        texts.append(extract_body(data["text"]))
X_train, X_test, y_train, y_test = train_test_split(
    texts, labels, test_size=0.3, random_state=42, stratify=labels
)

{"id": "76013", "theme": "misc.forsale", "text": "From: xorcist@cyberden.sf.ca.us\nSubject: [ For Sale ] EMS Freq. Shifter\n\nThis is yet for a friend again.\n\nEMS Freq. Shifter. (The machine that made those 3-D swirling guitar effects\n                    way back in the 70's.)\n\nSpacial panner with harmonic shifting. Very rare - A collectors Item. This\nis the last unit EMS ever made. Rack mountable. Mono in, Up/Down signals out\nwith seperate Pan out and sine/unsine voltage outs. VCO input too to control\nLFO. If interested contact Kevin before 9 pm PST (California) at 818-362-7883\nand make an offer.\n\nDo not reply to this account.\n\nHave a nice day\n\n__________________________________________________________________________\n   |       /         |\\\n   | H E   \\ Y B E R |/ E N            [ xorcist@cyberden.sf.ca.us ]\n\n   The CyberDen - Public Access Waffle Usenet System - 415/472-5527"}

['sale', 'em', 'freq', 'shifter', 'yet', 'friend', 'em', 'freq', 'shifter', 'spacial'

In [ ]:
vectorizer = CountVectorizer(
    tokenizer=preprocess_text,   
    preprocessor=None,
)

## I. For the 20N dataset compare two classifiers NB and LR to identify the 20 different newsgroups.

### Create your own processing pipeline for the task and justify it

### Divide the dataset into training (60%), validation (10%) and test (30%).

### Train NB and LR using the following vector representations:

#### tf (counts) representation (sklearn: CountVectorizer).

#### tfidf representation (sklearn: TfidfVectorizer).